### Step 1 : Prepare the Documents

In [3]:
import json

with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [4]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

documents[1]

{'text': 'GitHub - DataTalksClub/data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

### Step 2: Create Embeddings using Pretrained Models

In [2]:
from sentence_transformers import SentenceTransformer

In [5]:
model = SentenceTransformer("all-mpnet-base-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
len(model.encode('This is a simple sentence'))
# this lens is important. Different model has different lens

768

In [8]:
documents[1]

{'text': 'GitHub - DataTalksClub/data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

In [9]:
#created the dense vector using the pre-trained model
operations = []
for doc in documents:
    # Transforming the title into an embedding using the model
    doc["text_vector"] = model.encode(doc["text"]).tolist()
    operations.append(doc)

In [10]:
operations[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'text_vector': [-0.034499265253543854,
  -0.0792464017868042,
  -0.04578522592782974,
  0.008983444422483444,
  -0.04323797672986984,
  -0.011313831433653831,
  0.020990891382098198,
  -0.021203873679041862,
  -0.049964237958192825,
  -0.006591456010937691,
  0.05323801934719086,
  -0.010289818979799747,
  0.016171885654330254,
  0.09608883410692215,
  0.011132367886602879,
  -0.

### Step 3: Setup ElasticSearch Connection

In [12]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

es_client.info()

ObjectApiResponse({'name': '22f2d97eff9c', 'cluster_name': 'docker-cluster', 'cluster_uuid': '8KZzToRfSGi2NUmD73Lb4Q', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

### Step 4: Create Mappings and Index

In [13]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
        }
    }
}

In [14]:
index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

#### Step 5: Add documents into index

In [15]:
for doc in operations:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

#### Step 6: Create end user query

In [16]:
search_term = "windows or mac?"
vector_search_term = model.encode(search_term)


In [17]:

query = {
    "field": "text_vector", # in database, which field they should be searching
    "query_vector": vector_search_term,
    "k": 5, # 5 nearest documents
    "num_candidates": 10000, # group of documents is going to look into
}

In [18]:
res = es_client.search(index=index_name, knn=query, source=["text", "section", "question", "course"])
res["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'yBK9rZAB6Dos9MdDwv0Z',
  '_score': 0.71479183,
  '_source': {'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully'}},
 {'_index': 'course-questions',
  '_id': '6hO9rZAB6Dos9MdD1AAo',
  '_score': 0.6107276,
  '_source': {'question': 'WSL instructions',
   'course': 'mlops-zoomcamp',
   'section': 'Module 1: Introduction',
   'text': 'If you wish to use WSL on your windows machine, here are the setup instructions:\nCommand: Sudo apt install wget\nGet Anaconda download address here. wget <download address>\nTurn on Docker Desktop WFree Download | AnacondaSL2\nCommand: git clone <github repository address>\nVSCODE on WSL\nJupyter: pip3 install jupyter\nAdded by Gregory Morris (gwm1980@gmail.com)\nAll in all softwares a

#### Step 7: Perform Semantic Search & Advanced Search

In [19]:
# Note: I made a minor modification to the query shown in the notebook here
# (compare to the one shown in the video)
# Included "knn" in the search query (to perform a semantic search) along with the filter  
# This is Semantic search
knn_query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000
}


In [27]:
# This is a key word search
response = es_client.search(
    index=index_name,
    query={
        # "match": {"section": "General course-related questions"},
        "bool": {
            "must": {
                "multi_match": {
                    "query": "windows or python?",
                    "fields": ["text", "question", "course", "title"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term":{
                        "course": "data-engineering-zoomcamp" # only serarch for this course
                    }
                }
        }
    },
    knn=knn_query,
    size=5
)

In [ ]:
# Normaly the research is between 0 and 1
# but, when apply advance functionnality it will scale, and then will over 1

In [29]:
# This is semantic search
response = es_client.search(
    index=index_name,
    query={
        "match": {"section": "General course-related questions"},
    },
    knn=knn_query,
    size=5,
    explain=True
)

In [30]:
response["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'yBK9rZAB6Dos9MdDwv0Z',
  '_score': 11.598243,
  '_source': {'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully',
   'section': 'General course-related questions',
   'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'text_vector': [-0.026965495198965073,
    -0.0006259257206693292,
    -0.016629504039883614,
    0.05285143107175827,
    0.05476531386375427,
    -0.03133983165025711,
    0.029942642897367477,
    -0.04808562621474266,
    0.04467548429965973,
    0.005839455407112837,
    0.016233131289482117,
    0.012001212686300278,
    -0.0312223918735981,
    0.01660064235329628,
    -0.04886900633573532,
    -0.06496303528547287,
    0.046434178948402405,
    -0.009297648444771767,
    -0.0642528235912323,
    -0.01373270619660616,
    -0.015976140275597572,
    0.00862955953925848,
    -0.0244790073484182